In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime, timedelta

In [7]:
df = pd.read_csv('C:/big18/dl-dev/dl-dev\project/변동률과 체결강도/000660변동률Vector.csv', encoding='cp949')

df['18'].value_counts()

18
0.0    65760
2.0    64024
1.0    57044
Name: count, dtype: int64

In [8]:
df = pd.read_csv('../원본주식데이터/000660.csv',encoding='cp949')

# df = df[['code','종가','매도량','매수량']]
df

,Unnamed: 0,날짜,시간,시가,고가,저가,종가,거래량,code,MA5,MA20,MA60,MA120,Upper_Band,Lower_Band,day_name,매도량,매수량
0,2022-08-19 09:01:00,20220819,901,97000,97000,96500,96800,99206,660,NaN,NaN,NaN,NaN,NaN,NaN,Friday,11919.0,87281.0
1,2022-08-19 09:02:00,20220819,902,96800,97200,96700,97000,55648,660,NaN,NaN,NaN,NaN,NaN,NaN,Friday,15861.0,39787.0
2,2022-08-19 09:03:00,20220819,903,97100,97100,96200,96200,37596,660,NaN,NaN,NaN,NaN,NaN,NaN,Friday,17740.0,19856.0
3,2022-08-19 09:04:00,20220819,904,96200,96400,96100,96300,28194,660,NaN,NaN,NaN,NaN,NaN,NaN,Friday,15239.0,12955.0
4,2022-08-19 09:05:00,20220819,905,96300,96500,96100,96400,19232,660,96540.0,NaN,NaN,NaN,NaN,NaN,Friday,8216.0,11016.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
186835,2024-08-19 15:16:00,20240819,1516,193100,193200,193100,193200,11360,660,193140.0,192995.0,193810.00,194504.17,193370.44,192619.56,Monday,7343.0,4017.0
186836,2024-08-19 15:17:00,20240819,1517,193200,193200,192900,193000,32191,660,193120.0,193005.0,193778.33,194476.67,193369.05,192640.95,Monday,26959.0,5232.0
186837,2024-08-19 15:18:00,20240819,1518,192900,193000,192800,192800,9585,660,193040.0,193000.0,193746.67,194448.33,193372.77,192627.23,Monday,6745.0,2840.0
186838,2024-08-19 15:19:00,20240819,1519,192800,193100,192800,193000,16917,660,193020.0,193000.0,193713.33,194421.67,193372.77,192627.23,Monday,4646.0,12271.0


In [16]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime, timedelta


df = pd.read_csv('../원본주식데이터/000660.csv',encoding='cp949')
df = df[['code','종가','매수량','매도량']]

def divide_volumes(row):
    try:
        divisor = 728002365
        row['power'] = row['매수량'] - row['매도량']
        row['power'] = row['power']*1000000 / divisor
    except KeyError:
        print(f"코드 {row['code']}를 찾지 못했습니다.")
        input()
    return row


df = divide_volumes(df)

# 데이터 로드
df=df[['code','종가', 'power']]
df.rename(columns={'종가' : 'close'}, inplace=True)

# Unique 코드 및 카운트 계산
counts = df['code'].value_counts()
counts_dict = counts.to_dict()

# 결과를 저장할 리스트
results = []

for key in tqdm(counts_dict.keys()):
    value = counts_dict[key]
    for i in range(value - 12):   # 예측 분 -1
        start_idx = counts_dict[key] - value + i
        end_idx = start_idx + 9  # 묶을 분 -1

        # 벡터화하여 연산
        vectorList = []
        # for col in ['close','power']:
        #     diff = (df[col].values[start_idx:end_idx] - df[col].values[start_idx + 1:end_idx + 1]) 
        #     vectorList.extend(round(d, 2) for d in diff)
        diff = (df['close'].values[start_idx:end_idx] - df['close'].values[start_idx + 1:end_idx + 1]) / df['close'].values[start_idx:end_idx] * 10000
        vectorList.extend(round(d, 4) for d in diff)
        diff = (df['power'].values[start_idx:end_idx] - df['power'].values[start_idx + 1:end_idx + 1])
        vectorList.extend(round(d, 2) for d in diff)

        openValue = df['close'][start_idx + 12] - df['close'][start_idx + 9]
        if openValue > 0:
            vectorList.append(2.0)
        elif openValue == 0:
            vectorList.append(1.0)
        else:
            vectorList.append(0.0)

        results.append(vectorList)

# 결과를 DataFrame으로 변환
results_df = pd.DataFrame(results)

# CSV 파일에 저장
results_df.to_csv('./005930변동률Vector.csv', index=False)

100%|██████████| 1/1 [00:24<00:00, 24.56s/it]
